In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 72.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.7 MB

In [4]:
!pip install bertopic[visualization]

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-w3d_l8w8/kobert-tokenizer_fe3c0c54897d4711b29a70286031358a
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-w3d_l8w8/kobert-tokenizer_fe3c0c54897d4711b29a70286031358a
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=c1c0fbf2aee0faaabfc5ee10650ec5fba1b53757ac7559aa940d4be6f0d62822
  Stored in directory: /tmp/pip-ephem-wheel-cache-88bv134o/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [7]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bertopic import BERTopic
import os
import json

In [15]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    # zip 파일 압축 해제
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        file_path = os.path.join(extract_path, name)
        if os.path.isdir(file_path):
            for name in os.listdir(path=file_path):
                with open(file_path + '/' + name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    valid = []
                    for d in data['content'].split('\n'):
                        res = []
                        for word in d.split(' '):
                            if cond.match(word) != None:
                                res.append(word)
                        d = ' '.join(res)
                        if d and not d.replace(' ', '').isdecimal():
                            valid.append(d)
                    result.append('\n'.join([data['title']] + valid))
        else:
            with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
                data = json.load(f)
                valid = []
                for d in data['content'].split('\n'):
                    res = []
                    for word in d.split(' '):
                        if cond.match(word) != None:
                            res.append(word)
                    d = ' '.join(res)
                    if d and not d.replace(' ', '').isdecimal():
                        valid.append(d)
                result.append('\n'.join([data['title']] + valid))
    return result


In [16]:
import random
random.seed(42)
# galleries = ['ktwiz','hanwhaeagles_new','skwyverns_new1','ncdinos','samsunglions_new','doosanbears_new1','giants_new2','sh_new','lgtwins_new','tigers_new']
galleries = ['kart', 'kartriderplus', 'kartriderdrift']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen
cocktail[:5]

['근데 카드립 성패를 떠나서 만든다고 했을때부터 엥스럽지 않았냐\n그때가 그랬는데\n카트 거의 산송장급이고 섭종직전보다 사람 없었을땐데 카트 후속작 만든다길래\n이새끼들 병신인가? 생각부터 들었음',
 '블랙리스트 미드오픈팀이냐 ㅋㅋㅋㅋㅋ',
 '요즘넥슨디렉터 다친송받고있네\n박정무 강원기 마비노기디렉터까지 카트디렉터뺴고',
 '디시인사이드 유저 일동은 남자 연예인 갤러리의 폐쇄를 강력하게 요청합니다',
 '밥지훈 피셜 락스전 패배원인\n락스가 만만하노?ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ']

In [17]:
print(cocktail[0])

근데 카드립 성패를 떠나서 만든다고 했을때부터 엥스럽지 않았냐
그때가 그랬는데
카트 거의 산송장급이고 섭종직전보다 사람 없었을땐데 카트 후속작 만든다길래
이새끼들 병신인가? 생각부터 들었음


In [18]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split(' ') if word not in w.lower()])
stopwords = ['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']

In [19]:
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [20]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)

In [21]:
custom_tokenizer = CustomTokenizer(tokenizer)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
# vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [22]:
bertopic_model = BERTopic(language='multilingual',
                          vectorizer_model=vectorizer,
                          nr_topics=100,
                          top_n_words=10,
                          calculate_probabilities=True)

In [23]:
topics, probs = bertopic_model.fit_transform(cocktail)

In [24]:
bertopic_model.visualize_topics()

In [25]:
bertopic_model.visualize_distribution(probs[0])

In [26]:
from sklearn.cluster import KMeans

n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_


In [27]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
from sklearn.metrics import confusion_matrix
print(len(real_labels))
print(confusion_matrix(real_labels, labels))


19726
[[3477 6888  585]
 [ 743 2708   82]
 [1103 3971  169]]


In [28]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

def calc_purity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    _, col_ind = linear_sum_assignment(cm, maximize=True)
    reordered_cm = cm[:, col_ind]
    purity = np.sum(np.max(reordered_cm, axis=0)) / np.sum(reordered_cm)
    return purity, reordered_cm

purity, reordered_cm = calc_purity(real_labels, labels)

print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

Purity:  0.5551049376457468
Reordered Confusion Matrix: 
 [[6888  585 3477]
 [2708   82  743]
 [3971  169 1103]]


In [29]:
def show_cluster_examples(data, labels, n_clusters, examples_per_cluster=3):
    for cluster in range(n_clusters):
        print(f"Cluster {cluster + 1}:")

        cluster_indices = np.where(labels.astype(int) == cluster)[0]

        # Get examples from the cluster
        cluster_examples = [data[idx] for idx in cluster_indices]

        # Show example items
        for example in cluster_examples[:examples_per_cluster]:
            print(example)
        print()

n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_

# Show cluster examples
show_cluster_examples(cocktail, labels, n_clusters)


Cluster 1:
블랙리스트 미드오픈팀이냐
밥지훈 피셜 락스전 패배원인
락스가
에결가서 문호준이 지면 어떻하냐

Cluster 2:
카드립 성패를 떠나서 만든다고 했을때부터 엥스럽지 않았냐
그때가 그랬는데
카트 거의 산송장급이고 섭종직전보다 사람 없었을땐데 카트 후속작 만든다길래
이새끼들 병신인가? 생각부터 들었음
요즘넥슨디렉터 다친송받고있네
박정무 강원기 마비노기디렉터까지 카트디렉터뺴고
디시인사이드 유저 일동은 남자 연예인 갤러리의 폐쇄를 강력하게 요청합니다

Cluster 3:
대 호 준
카갤놈들아
안녕
라온아
니가 준 싱카 막힌다고 했나?

